### Loading a CSV file

In [1]:
df_data = pd.read_csv(body)
df_data.head()


,Unnamed: 0,label,text,label_num
0,605,ham,Subject: enron methanol ; meter # : 988291\r\n...,0
1,2349,ham,"Subject: hpl nom for january 9 , 2001\r\n( see...",0
2,3624,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar...",0
3,4685,spam,"Subject: photoshop , windows , office . cheap ...",1
4,2030,ham,Subject: re : indian springs\r\nthis deal is t...,0


In [2]:
df_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5171 entries, 0 to 5170
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  5171 non-null   int64 
 1   label       5171 non-null   object
 2   text        5171 non-null   object
 3   label_num   5171 non-null   int64 
dtypes: int64(2), object(2)
memory usage: 161.7+ KB


In [3]:
df_data.shape

(5171, 4)

### Droping the columns which are not required

In [4]:
df = df_data.drop(['Unnamed: 0','label_num'],1)

In [5]:
df.head()

,label,text
0,ham,Subject: enron methanol ; meter # : 988291\r\n...
1,ham,"Subject: hpl nom for january 9 , 2001\r\n( see..."
2,ham,"Subject: neon retreat\r\nho ho ho , we ' re ar..."
3,spam,"Subject: photoshop , windows , office . cheap ..."
4,ham,Subject: re : indian springs\r\nthis deal is t...


### Mapping 'ham' to 0 and 'spam' to 1

In [6]:
import numpy as np
df.label = np.where(df.label=='ham',0,1)
df.head()

,label,text
0,0,Subject: enron methanol ; meter # : 988291\r\n...
1,0,"Subject: hpl nom for january 9 , 2001\r\n( see..."
2,0,"Subject: neon retreat\r\nho ho ho , we ' re ar..."
3,1,"Subject: photoshop , windows , office . cheap ..."
4,0,Subject: re : indian springs\r\nthis deal is t...


### Using sub() method of regular expression class for removing punctuations

In [7]:
import re
df.text=df.text.apply(lambda x: re.sub(r'[^\w\s]','',x))
df.text=df.text.apply(lambda x: x.lower())
df.head()

,label,text
0,0,subject enron methanol meter 988291\r\nthis...
1,0,subject hpl nom for january 9 2001\r\n see at...
2,0,subject neon retreat\r\nho ho ho we re aroun...
3,1,subject photoshop windows office cheap mai...
4,0,subject re indian springs\r\nthis deal is to ...


#### Use nltk to remove stopwords

import nltk

from nltk.corpus import stopwords

nltk.download('stopwords')

def rm_word(text): 

    clean_words = [char for char in text.split() if char.lower() not in stopwords.words('english')] 
    
    clean_words = ''.join(clean_words)
    
    return clean_words

df['text'] = df['text'].apply(rm_word)

* By removing stop words my model's score is 0.719

## Model Building

In [8]:
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import MultinomialNB
from sklearn.feature_extraction.text import CountVectorizer    #bag of words
from sklearn.pipeline import Pipeline

In [9]:
x_train,x_test,y_train,y_test=train_test_split(df.text,df.label,test_size=0.2,random_state=0)

In [10]:
model =Pipeline([('vectorizer',CountVectorizer()),('nb',MultinomialNB())])
model.fit(x_train,y_train)

Pipeline(steps=[('vectorizer', CountVectorizer()), ('nb', MultinomialNB())])

### Model Score

In [11]:
model.score(x_test,y_test)

0.9729468599033816

### Using predict method of model to classify the mails

In [12]:
model.predict(x_test)

array([0, 0, 0, ..., 0, 1, 0])

In [13]:
model.predict(['you won 10 dollars'])

array([1])

In [14]:
def spam_pred(x):
    if model.predict([x])==0:
        print('H A M')
    else:
        print('**S P A M**')

In [15]:
spam_pred('you win 10 dollars')

**S P A M**


In [16]:
spam_pred('hi viprav, how are you? when you are coming for a lunch?')

H A M
